In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf

from Functions.conn import db_upload
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model
from sklearn.preprocessing import minmax_scale
print(tf.__version__)


C:\Users\brian\Anaconda3\envs\a3d\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\brian\Anaconda3\envs\a3d\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\brian\Anaconda3\envs\a3d\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\brian\Anaconda3\envs\a3d\lib\site-packages\tensorflow\python\framework

2.0.0-alpha0


In [22]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="a3d"
)

def db_upload(sql, val):
  mycursor = mydb.cursor()
  mycursor.execute(sql, val)
  mydb.commit()
  print(mycursor.rowcount, "record inserted.")


verbosity = 1
groupSize = 288
days_indicator=30 #set equal to t+X days indicator

index_names = [ 'Date', 'ASA51', 'XNDX', 'SPXT', 'CACR', 'DAX', 'NKYTR', 'HSI1', 'KSP2TR']

subset = [ 'ASA51', 'XNDX', 'SPXT', 'CACR', 'DAX', 'NKYTR', 'HSI1', 'KSP2TR' ]

#note, raw.csv omits 'date' label in cell A1 of csv but incl's all index names in A2++ so csv can be read more easily but re "dropna"
#rule below, the header row is omitted from all program calculations
index_data = pd.read_csv( 'raw.csv', names = index_names ) # read file, insert header row
index_data.dropna( inplace = True ) # this removes rows with >=1 #na, being 234/1,304 data points in v1 dataset, inplace= True, 'do operation inplace and return None.''
# index_data.describe() #prints some table metrics, not part of any calculation hereafter.
print("index_data shape is ")
print(index_data.shape)
index_data.to_csv('out.csv', index=False)


ASAMin=0
ASAMax=0
ASArange=0

#function to create trainingArray that groups data into lists of 'groupSize' date rows (atm 288) * all indices' data (8)
#creating a master list of 780 rows of 2304 columns, essentially 780 moving windows of all data
def groupData( data, groupSize ):
	trainingArray = []
	for i in range( data.size ):
		if i >= groupSize and i <= (data.shape[0]-days_indicator) :
			trainingListSingleEntry = []
			for ii in range( groupSize ):
				trainingListSingleEntry = np.concatenate( ( trainingListSingleEntry, data[ i - ii -1] ), axis=None )
			trainingArray.append( trainingListSingleEntry )
	return trainingArray


def groupLabels( data, groupSize ):
	trainingArray = []
	for i in range( data.size):
		if i >= (groupSize+days_indicator-1):
			trainingListSingleEntry = data[i]
			trainingArray.append( trainingListSingleEntry )
	return trainingArray

def doTraining():
	trainingData = index_data[ subset ].values
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	trainingData = np.array( trainingData )
# 	print("training data 1", trainingData[0])
	print("trainingData groupData shape expecting 752 x 2304", trainingData.shape)

#trainingLabels being target, output dataset
	trainingLabels = index_data[ 'ASA51' ].values
	trainingLabels = minmax_scale( trainingLabels )
# 	print("training label size is ")
# 	print(trainingLabels.size)
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
	trainingLabels = np.array( trainingLabels )
	print("training label row 1",trainingLabels[0])
	print("training label shape expecting 753 * 1 ", trainingLabels.shape)

	model = Sequential()
	print("input layer set to:", len( trainingData[0] ))
	model.add( Dense( 32, input_dim = len( trainingData[0] ), kernel_initializer = 'normal', activation = 'relu' ) ) # going into a 32-unit layer
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) ) # Another hidden layer of 16 units
	model.add( Dense( 32, kernel_initializer = 'normal', activation = 'relu' ) ) # Another hidden layer of 16 units
	model.add( Dense( 1 ) ) # Output layer with a binary classification ( Democrat or Republican )
	model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] ) # Compile model
	model.fit( trainingData, trainingLabels, epochs = 1, batch_size = 50, verbose = verbosity ) # Train model
	scores = model.evaluate( trainingData, trainingLabels, verbose = verbosity ) # Grade the model
	print( "%s: %.2f%%" % ( model.metrics_names[1], scores[1]*100 ) )
	model.save( 'BHMarket_Model.h5' ) # Save the model
# 	plot_model(model, to_file='modelplotbh.png')
    
def doPrediction():
	trainingData = index_data[ subset ].values
	print("trainingData groupData shape expecting unsure", trainingData.shape)
	ASAmin =  float((trainingData.min(axis=0))[0])
	print("min in trainingData is: ", ASAmin)
	ASAmax =  float((trainingData.max(axis=0))[0])
	print("max in trainingData is: ", ASAmax)
	ASArange =  ASAmax-ASAmin
	print("trainingData groupData shape expecting unsure", trainingData.shape)
	trainingData = minmax_scale( trainingData )
	trainingData = groupData( trainingData, groupSize = groupSize )
	inputData = trainingData[-1]
	print( "input data is: ", inputData )

	trainingLabels = index_data[ 'Date' ].values
	trainingLabels = groupLabels( trainingLabels, groupSize = groupSize )
# 	date = trainingLabels[-1]
# 	print( "training data shape is", inputData.shape )

	loaded_model = load_model( 'BHMarket_Model.h5' )

	sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
	# loaded_model = load_model( 'LSMarket_Model.h5' )
	# evaluate loaded model on test data
	loaded_model.compile( loss = 'mse', optimizer = 'rmsprop', metrics = [ 'mae' ] )
	# Predict things...
	print( "input data shape is: ", inputData.shape )
	prediction_1 = loaded_model.predict( inputData.reshape( (1, 2304) ), batch_size = None, verbose = verbosity, steps = None )
	print("+30 day prediction scaled is:", prediction_1)    
	prediction_1=prediction_1*ASArange+ASAmin
	print("+30 day prediction is:", prediction_1)
	x123=prediction_1.tolist()
	final_int=x123[0]
	print(final_int)
	print("final_int variable is type", type(final_int))
	val = ("delta_strategy", final_int)
	db_upload(sql,val)
    
doTraining()


print("prediction is: ")
doPrediction()




sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
zzz=12
val = ("delta_strategy", zzz)
db_upload(sql,val)

index_data shape is 
(1070, 9)
trainingData groupData shape expecting 752 x 2304 (753, 2304)
training label row 1 0.10207420407148349
training label shape expecting 753 * 1  (753,)
input layer set to: 2304
753/753 [==============================] - 0s 116us/sample - loss: 0.0307 - mae: 0.1395
mae: 13.95%
prediction is: 
trainingData groupData shape expecting unsure (1070, 8)
min in trainingData is:  43542.09
max in trainingData is:  65901.51
trainingData groupData shape expecting unsure (1070, 8)
input data is:  [0.88022185 0.86066795 0.87067035 ... 0.61695442 0.6628082  0.77504406]
input data shape is:  (2304,)
1/1 [==============================] - 0s 11ms/sample
+30 day prediction scaled is: [[1.0806971]]
+30 day prediction is: [[67705.85]]
[67705.8515625]
final_int variable is type <class 'list'>


ProgrammingError: Failed processing format-parameters; Python 'list' cannot be converted to a MySQL type

In [ ]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="a3d"
)

def db_upload(sql, val):
  mycursor = mydb.cursor()
  mycursor.execute(sql, val)
  mydb.commit()
  print(mycursor.rowcount, "record inserted.")


sql = "INSERT INTO daily_ai_indicators (metric, value) VALUES (%s, %s)"
val = ("delta_strategy", 123477)
db_upload(sql,val)

In [4]:
abc=1
print("type is: ", type(abc))

type is:  <class 'int'>


In [14]:
print(x123)

NameError: name 'x123' is not defined